In [39]:
import sys
!{sys.executable} -m pip install xgboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [40]:
df = pd.read_csv('train.csv')
columns = list(df.columns)
non_medical = columns[0:79]
medical = columns[79:127]

med  = df[medical]
med = med.sum(axis = 1)

df['Product_Info_2'] = pd.Categorical(df['Product_Info_2'])
dfDummies = pd.get_dummies(df['Product_Info_2'], prefix = 'P2')

train = df[non_medical]
train = train.drop(columns = 'Product_Info_2')
train['Response'] = df['Response']
train['Keyword'] = med
train = pd.concat([train, dfDummies], axis=1)
train.head

<bound method NDFrame.head of           Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0          2               1              10        0.076923               2   
1          5               1              26        0.076923               2   
2          6               1              26        0.076923               2   
3          7               1              10        0.487179               2   
4          8               1              26        0.230769               2   
5         10               1              26        0.230769               3   
6         11               1              10        0.166194               2   
7         14               1              26        0.076923               2   
8         15               1              26        0.230769               2   
9         16               1              21        0.076923               2   
10        17               1              26        0.128205               2   
11        

In [41]:
train.Keyword

0        0
1        0
2        0
3        1
4        0
5        2
6        0
7        0
8        1
9        2
10       4
11       1
12       1
13       1
14       2
15       3
16       1
17       0
18       1
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       1
27       0
28       2
29       2
        ..
59351    0
59352    1
59353    3
59354    1
59355    1
59356    2
59357    0
59358    0
59359    1
59360    1
59361    0
59362    0
59363    0
59364    1
59365    0
59366    2
59367    1
59368    0
59369    6
59370    0
59371    1
59372    4
59373    0
59374    0
59375    1
59376    0
59377    0
59378    1
59379    2
59380    0
Name: Keyword, Length: 59381, dtype: int64

In [42]:
df = pd.read_csv('test.csv')
columns = list(df.columns)
non_medical = columns[0:79]
medical = columns[79:127]

med  = df[medical]
med = med.sum(axis = 1)

df['Product_Info_2'] = pd.Categorical(df['Product_Info_2'])
dfDummies = pd.get_dummies(df['Product_Info_2'], prefix = 'P2')

test = df[non_medical]
test = test.drop(columns = 'Product_Info_2')
test['Keyword'] = med
test = pd.concat([train, dfDummies], axis=1)

In [43]:
clf = LogisticRegression(solver='lbfgs', multi_class='multinomial',
                           random_state=1, max_iter = 100)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4 = AdaBoostClassifier(n_estimators=100, random_state=0)
clf5 = DecisionTreeClassifier(max_depth=20, min_samples_split=20,
     random_state=0)
clf6 = ExtraTreesClassifier(n_estimators=100, max_depth=20,
     min_samples_split=20, random_state=0)
clf7 = XGBClassifier()

In [44]:
clf1 = VotingClassifier(estimators=[
         ('lr', clf), ('rf', clf2), ('gnb', clf3), ('ada', clf4), ('5', clf5), ('6', clf6), ('7', clf7)], voting='hard')

In [45]:
X = train
X = X.drop(columns  = 'Id')
y = X['Response']
X = X.fillna(-1)

X = X.drop(columns = 'Response')

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=0)

scaler = StandardScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
clf1 = clf1.fit(X_train, y_train)
print(clf1.score(X_train,y_train))

In [ ]:
predictions_test = clf1.predict(X_test)
print(clf1.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
CM = confusion_matrix(y_test, predictions_test)
pritn(CM)
print(classification_report(y_test, predictions_test))

In [ ]:
test_noID = test.drop(columns = ['Id'])
test_noID = test_noID.fillna(-1)

scaler = StandardScaler()
scaler.fit(test_noID)
test_noID = scaler.transform(test_noID)
predictions_test = clf1.predict(test_noID)
        
test['Response'] = predictions_test
submission = test[['Id', 'Response']]
submission.set_index('Id', inplace = True)
submission.to_csv('Submission.csv', float_format='%.0f')
print(submission)